In [46]:
import cv2
import time
import json

In [165]:
cap = cv2.VideoCapture(0)

In [151]:
while True:
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #print(rgb)

    obuff=cv2.resize(rgb, (64,64))
    #cv2.thresh(obuff, )
    #ret,thresh4 = cv2.threshold(obuff,127,255,cv2.THRESH_TOZERO)

    nobuff = bytearray(int(64*64/8))
    cbit = 0
    cpos = 0
    for i in range(64):
        for j in range(64):
            bout = obuff[i][j] > 200
            if (bout):
                nobuff[cbit] |= 1 << cpos
            cpos += 1
            if cpos >= 8:
                cpos = 0
                cbit += 1
    print(nobuff)

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
        off += 64
        #print((lensbuff)

        ln = 0#arduino.write(sbuff)
        print('%s' % ln)
        time.sleep(0.1)
    print('-'*80)    
    cv2.imshow('frame', obuff)
    time.sleep(2)
    #break


bytearray(b"\x00\x00\x00\x80\xf8\xff\xff\x1f\x00\x00\x00\x00\xf0\xff\xff\x0f\x00\x00\x00\x00\xf2\xff\xff\x1f\x00\x00\x00\x00\xe0\xff\xff\x07\x00\x00\x00\x00\xc0\xff\xff\'\x00\x00\x00@\xc0\xff\xff?\x00\x00\x00p\x80\xff\xff\x1f\x00\x00\x00x\xc0\xff\xff\x03\x00\x00\x008\x80\xff\xff\x01\x00\x00\x00<\x80\xff\xff\x00\x00\x00\x00\x1c\x80\xff\xff\x00\x00\x00\x00\x0e\x80\xff\x7f\x00\x00\x00\x00\n\x00\xff\x7f\x00\x00\x00\x00\x00@\xff\x7f\x00\x00\x00\x80\x00@\xff\x7f\x00\x00\x00\x00\x00\x00\xff\xff\x01\x00\x00\x00\x00\x00\xfc\xff\x00\x00\x00\x00\x00\x80\xf8\xc7\x00\x00\x00\x00\x00\x80\xf0\x03\x00\x00\x00\x00\x00\x80\xf0\x01\x00\x00\x00\x00\x00\x00\xe0\x00\x00\x00\x00\x00\x00\x00\xe0\x00\x00\x00\x00\x00\x00\x00\xe0\x00\x00\x00\x00\x00\x00\x00\xe0\x00\x00\x00\x00\x00\x00\x80`\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00

KeyboardInterrupt: 

In [179]:
cv2.destroyAllWindows()

In [186]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

[ 84 101 108 255]


In [176]:
#cap.release()
import random

In [192]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


KeyboardInterrupt: 

In [89]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [90]:
buff=bytearray(65)

In [91]:
arduino.write(b'\x04'+buff)

66

In [107]:
for i in range(65):
    buff[i] = i

In [108]:
for j in range(10):
    for i in range(10):
        buff[0] = i
        ln = arduino.write(buff)
        print('%s' % ln)
        time.sleep(0.1)


65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65


In [94]:
while True:
    print(arduino.readline())

b'\x00\x80\x00\x80\x80\x00\x80\x00\x80\x80\x00\x80\x80\x00\x00\x80\x80\x00\x80\x00\x00\x00\x00\x00\x80\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x80\x00\x00\x80\x00\x80\x80\x80\x00\x00\x80\x00\xfe\x80\x00\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x80\x80\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x00\x00\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x80\x00\x80\x80\x80\x00\x00\x80\x00\x00\x00\x80\x80\x00\x80\x00\x00\x80\x80\x80\x00\x80\x80\x80\x00\x00\x80\x00\x80\x00\x80\x80\x80\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x00\x00\x00\x80\x80\x80\x80\x80\x00\x00\x00\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x00\x80\x80\x80\x80\x00\x80\x80\x80\x00\x00\x80\x0065\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65

KeyboardInterrupt: 